<a href="https://colab.research.google.com/github/dudaholandah/NLP/blob/main/Projeto/Text_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!cp -r '/content/drive/MyDrive/Datasets/cuisine-classification-ingredients' 'cuisine-classification-ingredients'

In [3]:
!cp -r '/content/drive/MyDrive/Datasets/indian-food' 'indian-food'

# Imports

In [ ]:
!pip install -U sentence-transformers

In [ ]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
#WORD2VEC
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
#BERT
import tensorflow as tf
from sentence_transformers import SentenceTransformer
#Data
import json
import os
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# Data

Cuisine Classification Data

In [6]:
path = 'cuisine-classification-ingredients/train.json'

with open(path, 'r') as f:
  json = json.load(f)

In [7]:
df_cuisine = pd.DataFrame(json, columns=['id', 'cuisine', 'ingredients'])

In [8]:
df_cuisine = df_cuisine.drop(['id'], axis=1)

In [9]:
df_cuisine.head(5)

,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Indian Food Data

In [10]:
df_indian = pd.read_csv('indian-food/indian_food.csv')

In [11]:
df_indian.head(5)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


## Pre-Processing

In [12]:
def pre_processing(text):
  new = ""
  for word in text.split(','):
    word = re.sub(r'[^\w\s]', '', word.lower())
    word = re.sub(r'[0-9]+', '', word)
    word = re.sub(r'\boz\b', '', word)
    word = re.sub(r'\ba taste of\b', '', word)
    new = new + word.strip() + " "

  return new[:-1]

Cuisine Classification Data

In [13]:
df_cuisine['ingredients'] = [','.join(x).strip() for x in df_cuisine['ingredients']]

In [14]:
df_cuisine['ingredients'] = [pre_processing(x) for x in df_cuisine['ingredients']]

In [15]:
df_cuisine.head(5)

,cuisine,ingredients
0,greek,romaine lettuce black olives grape tomatoes ga...
1,southern_us,plain flour ground pepper salt tomatoes ground...
2,filipino,eggs pepper salt mayonaise cooking oil green c...
3,indian,water vegetable oil wheat salt
4,indian,black pepper shallots cornflour cayenne pepper...


Indian Food Data

In [16]:
df_indian['ingredients'] = [pre_processing(x) for x in df_indian['ingredients']]

In [17]:
df_indian.head(5)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,maida flour yogurt oil sugar,vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,gram flour ghee sugar,vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,carrots milk sugar ghee cashews raisins,vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,flour ghee kewra milk clarified butter sugar a...,vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,milk powder plain flour baking powder ghee mil...,vegetarian,15,40,sweet,dessert,West Bengal,East


# Different Embeddings

## TFIDF

Cuisine Classification Data

In [33]:
tfidf_vectorizer = TfidfVectorizer()

In [34]:
tfidf_cuisine = tfidf_vectorizer.fit_transform(df_cuisine['ingredients'])

In [35]:
tfidf_cuisine = tfidf_cuisine.toarray()

In [36]:
df_tfidf_cuisine = pd.DataFrame(tfidf_cuisine, columns=tfidf_vectorizer.get_feature_names_out())

In [37]:
df_tfidf_cuisine.head(5)

,abalone,abbamele,absinthe,abura,acai,accent,accompaniment,achiote,acid,acini,...,yuzukosho,zaatar,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Indian Food Data

In [38]:
tfidf_indian = tfidf_vectorizer.fit_transform(df_indian['ingredients'])

In [39]:
tfidf_indian = tfidf_indian.toarray()

In [40]:
df_tfidf_indian = pd.DataFrame(tfidf_indian, columns=tfidf_vectorizer.get_feature_names_out())

In [41]:
df_tfidf_indian.head(5)

,acid,all,almond,almonds,aloo,alum,amaranth,amchur,and,anise,...,whipping,white,whole,wild,wine,yam,yellow,yoghurt,yogurt,yolks
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.540083,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.349811,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


## Word2Vec

In [18]:
def get_embedding_w2v(model, doc_tokens):
  embeddings = []
  for tok in doc_tokens:
    if tok in model.wv.vocab:
      embeddings.append(model.wv.word_vec(tok))
    else:
      embeddings.append(np.random.rand(300))
  return np.mean(embeddings, axis=0)

Cuisine Classification Data

In [19]:
ing_cuisine = '. '.join(df_cuisine['ingredients'])
word2vec_cuisine = nltk.sent_tokenize(ing_cuisine)
word2vec_cuisine = [nltk.word_tokenize(sentence) for sentence in word2vec_cuisine]

In [20]:
model_word2vec = Word2Vec(word2vec_cuisine, size=300, min_count=1, window=5, sg=1,workers=4)

In [21]:
word2vec_cuisine = df_cuisine['ingredients'].apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

Indian Food Data

In [22]:
ing_indian = '. '.join(df_indian['ingredients'])
word2vec_indian = nltk.sent_tokenize(ing_indian)
word2vec_indian = [nltk.word_tokenize(sentence) for sentence in word2vec_indian]

In [ ]:
model_word2vec = Word2Vec(word2vec_indian, size=300, min_count=1, window=5, sg=1,workers=1)

In [24]:
word2vec_indian = df_indian['ingredients'].apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

## BERT

In [ ]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [47]:
bert_cuisine = bert_model.encode(df_cuisine['ingredients'][:1000].values)

In [46]:
bert_indian = bert_model.encode(df_indian['ingredients'].values)

# Different Classificators

# Text Retrieval

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
def text_retrieval(dataset, embedding):
  top10 = [0] * 10
  size = len(dataset)

  for i in range(size):
    similarity = cosine_similarity([embedding[i]], embedding[:size])
    similarity = [(similarity, idx) for idx,similarity in enumerate(similarity[0])]

    similarity.sort(reverse=True)
    top = similarity[1:11]

    cat_orig = dataset.iloc[i]

    for idx, [sim, j] in enumerate(top):
      top10[idx] += cat_orig in dataset.iloc[j]

  return top10

## TFIDF

In [51]:
cuisine = df_cuisine['cuisine']
indian = df_indian['diet']

In [42]:
top10_tfidf_cuisine = text_retrieval(cuisine, tfidf_cuisine)
top10_tfidf_indian = text_retrieval(indian, tfidf_indian)

In [43]:
top10_tfidf_cuisine

[543, 467, 486, 461, 431, 427, 424, 417, 425, 387]

In [44]:
top10_tfidf_indian

[240, 239, 230, 232, 235, 234, 231, 230, 232, 237]

In [ ]:
top10_word2vec_cuisine = text_retrieval(cuisine, word2vec_cuisine.tolist())
top10_word2vec_indian = text_retrieval(indian, word2vec_indian.tolist())

In [31]:
top10_word2vec_cuisine

[597, 555, 536, 539, 504, 504, 507, 491, 466, 461]

In [32]:
top10_word2vec_indian

[234, 234, 234, 232, 233, 227, 231, 232, 230, 229]

In [48]:
top10_bert_cuisine = text_retrieval(cuisine, bert_cuisine)
top10_bert_indian = text_retrieval(indian, bert_indian)

In [49]:
top10_bert_cuisine

[380, 349, 317, 275, 256, 266, 255, 247, 246, 227]

In [50]:
top10_bert_indian

[239, 237, 239, 244, 238, 234, 239, 237, 233, 236]